In [ ]:
!pip install --upgrade google-api-python-client

In [ ]:
import sys
import yaml
import glob
import os
import pandas as pd
import time

from random import randint
# pip install --upgrade google-api-python-client
from googleapiclient import discovery

In [ ]:
credentials = yaml.load(open("../credentials.yaml"))["perspective-api"]

In [ ]:
def get_toxicity_score(service, text):
    try:
        if sys.getsizeof(text) > 0 and sys.getsizeof(text) < 3000:
            analyze_request = {
                'comment': {'text': text},
                'requestedAttributes': {'TOXICITY': {}}
            }
            response = service.comments().analyze(body=analyze_request).execute()
            toxicity_score = (
                response.get("attributeScores")
                .get("TOXICITY")
                .get("summaryScore")
                .get("value")
            )
        else:
            toxicity_score = None
    except:
        toxicity_score = None
    
    return toxicity_score

In [ ]:
service = discovery.build(
    "commentanalyzer",
    "v1alpha1",
    developerKey=credentials["key-1"],
    discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
    static_discovery=False,
)

In [ ]:
input_path = "../data/01_preprocessed"
output_path = "../data/02_scored"
filenames = [file.split("/")[-1] for file in glob.glob(os.path.join(input_path, "*"))]

for filename in filenames:
    df_sentences = pd.read_csv(os.path.join(input_path, filename))
    for i, row in df_sentences.iterrows():
        df_sentences.loc[i, "score"] = get_toxicity_score(service, row.text)
        time.sleep(1)

    df_sentences.to_csv(os.path.join(output_path, f"scored-{filename}.csv"), index=False)